In [ ]:
%env DDS_CACHE_DISABLE=False
%env DDSAPI_CLIENT_CACHE_DIR=/home/jovyan/pilot1/data/.cache
%env DDSAPI_RC=/home/jovyan/work/ddsapirc

In [ ]:
import ddsapi
import xarray as xr

### SIMPLE WORKFLOW
- use functions to define the different steps
- use parameters (so function can be reused for different inputs - especially input data)

#### Step1: Get input data

In [ ]:
def get_data(variables, area, time) -> xr.Dataset:
    c = ddsapi.Client()
    ds = c.retrieve("era5-downscaled-over-italy", "hourly",
        {
            "area": area,
            "time": time,
            "variable": variables,
            "format": "netcdf"
        })
    return ds

#### Step2: Process

In [ ]:
def process(ds) -> xr.Dataset:
    return ds.mean(dim=['rlat', 'rlon'])

#### Step3: Publish

In [ ]:
def publish(ds, filename) -> None:
    ds.to_netcdf(f'/home/jovyan/pilot1/results/{filename}')

#### Main Flow

In [ ]:
variables = [ "air_temperature", "precipitation_amount" ]
area = { "north": 44.1, "south": 40.1,  "west": 11.5, "east": 13.5 }
time = { "start": "2020-01-03T12:00", "stop": "2020-01-05T12:00" }

In [ ]:
ds = get_data(variables, area, time)

In [ ]:
processed_ds = process(ds)

In [ ]:
publish(processed_ds, 'average_t2m_prec.nc')

In [ ]:
xr.open_dataset('/home/jovyan/pilot1/results/average_t2m_prec.nc', decode_coords='all')